In [1]:
#Imports
from __future__ import unicode_literals, print_function, division
from io import open
import wget # to download passwordlist
import glob
import os
import numpy as np
import unicodedata
import string
import torch
import torch.nn as nn
from torch.autograd import Variable

Passwortdatei auslesen

In [2]:
# https://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
        and c in allowed_letters
    )

def readPasswords(filename):
    passwords = open(filename, encoding="utf8").read().split("\n")
    return [unicodeToAscii(password) for password in passwords]

def charToIndex(char):
    return allowed_letters.find(char)

def passwordToTensor(name):
    ret = torch.zeros(len(name), 1, password_len)
    for i, char in enumerate(name):
        ret[i][0][charToIndex(char)] = 1
    return ret

def targetToTensor(password):
    indizes = [letters.find(password[i]) for i in range(1,len(password))]
    indizes.append(password_len - 1)
    return torch.LongTensor(indizes)
                                                        
    
allowed_letters = string.ascii_letters + string.digits + string.punctuation
password_len = len(allowed_letters) + 1

Download der Passwortliste

In [3]:
#rockyou Liste produziert: 'utf-8' codec can't decode byte 0xf1 in position 5079963: invalid continuation byte
#filename = 'rockyou.txt'#Vermutung: Liste ist zu lang
filename = 'john.txt'

if os.path.isfile(filename):
    print('passwordlist exists')
    passwords = readPasswords(filename) #Pfad noch anpassen
else:
    print('download passwordlist:')
    #download list
    wget.download('https://www.scrapmaker.com/data/wordlists/dictionaries/rockyou.txt')
    passwords = readPasswords(filename) #Pfad noch anpassen

#entfernen von leeren Zeilen
passwords = [passw for passw in passwords if passw != '']
for passw in passwords:
    if len(passw)<1:
        print('pass:-'+passw+'-')

passwordlist exists


In [4]:
def prepareData():
    lines = []
    f = open(filename,encoding="utf-8").read().split('\n')
    for line in f:
        lines.append(line)
        
    return lines


Deklarierung der Variablen:

In [5]:
lstm_inputs = 10 # kann geaendert werden. 1 = 1 input pro timestep
hidden_layers = 32
batch_size = 10#Batch size noch berechnen
model_output = 1
model_layers = 2
iterations = 10000

LSTM-Generator:

In [6]:
class PasswordLSTMGenerator(nn.Module):
    def __init__(self, input_lstm, hiddens, output ):
        super(PasswordGenerator, self).__init__()
        self.input_len = input_lstm
        self.hiddens = hiddens
        self.output = output
        
        self.lstm()
    
    
    def __init__hiddens(self):
        hiddens = torch.zeros(self.layers, self.batch, self.hidden)
        return hiddens, hiddens
                                         
                          
    def forward_pass(self, input):
        output_lstm, self.hiddens = self.lstm(input.view(len(input), self.batch, -1))
        pred = self.linear(output_lstm[-1].view(self.batch,-1))
    
        return y_pred.view(-1)
    
model = PasswordLSTMGenerator(lstm_inputs, hidden_layers, batch= batch_size, output = model_output, layers = model_layers)

TypeError: __init__() got an unexpected keyword argument 'batch'

LSMT-Training:

In [ ]:
#Train
history = np.zeros(iterations)

for i in range(iterations):
    model.hidden = model.init_hidden()
    
    pred = model